In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import ast


## Evaluation for ER

In [ ]:
 CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/ER/wdc_all-MoE-Add --gpu_num 2 --file dataset/ER/wdc_all_test_output.json --max_token 32 --json

In [2]:
test_file_path = 'inference_Transfer_ER/wdc_all-MoE-Add--wdc_all_test_output.csv' ## replace with the inference file
result_merge = pd.read_csv(test_file_path,index_col=0).fillna('')
def Transfer(row):
    if(row['output'].__contains__('mismatch')) or (row['output'].__contains__('dismatch')):
        label = 0
    else:
        label = 1
    if(row['predict'].__contains__('mismatch')) or (row['predict'].__contains__('dismatch')):
        predict = 0
    else:
        predict = 1
    return label,predict
result_output = result_merge.apply(Transfer,axis=1,result_type='expand')
from sklearn.metrics import f1_score, precision_score,recall_score
print(precision_score(y_true=result_output[0],y_pred=result_output[1]),recall_score(y_true=result_output[0],y_pred=result_output[1]),f1_score(y_true=result_output[0],y_pred=result_output[1]))

0.9102455546147333 0.8973288814691152 0.9037410676754939


## Evaluation for Data Cleaning

- If the result of Error Detection changed, please change the `detector.npy` and test file, e.g. `dataset/DC/beer_test.json` correspondingly. Each detected error is related to one record in test file.

### Dataset Hospital

 CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/DC/rayyan_train-MoE-Add --gpu_num 2 --file dataset/DC/rayyan-test-20.json --max_token 512 --json

In [ ]:
 CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/DC/beer_train-MoE-Add --gpu_num 2 --file dataset/DC/beer_test.json --max_token 512 --json

In [ ]:
 CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/DC/hospital_train-MoE-Add --gpu_num 2 --file dataset/DC/hospital-test.json --max_token 512 --json

### Dataset Hospital

In [4]:
test_file_path = 'inference/hospital-test.csv' ## Replace the train file
hospital_result = pd.read_csv(test_file_path,index_col=0) ## 
count = 0
hospital_clean = pd.read_csv('raw_dataset/DC/hospital/original/clean.csv').astype(str)
hospital_dirty = pd.read_csv('raw_dataset/DC/hospital/original/dirty.csv').astype(str)
hospital_dirty.columns = hospital_clean.columns
hospital_correction = hospital_dirty.copy()
hospital_detector = np.load('raw_dataset/DC/hospital/detector/detector.npy').reshape((-1,20))
import ast
for d in np.argwhere(hospital_detector==1):
    i = d[0]
    j = d[1]
    try:
        predict = list(eval(hospital_result.iloc[count,-1]).values())[0]
        hospital_correction.iloc[i,j] = predict
        count += 1
    except:
        predict = hospital_result.iloc[count,-1]
        hospital_correction.iloc[i,j] = predict
        count += 1
# print(count,len(hospital_result))
All_Data_Error = 0
All_Fixed_Error = 0
Correct_Fixed_Error = 0
clean = hospital_clean.copy()
dirty = hospital_dirty.copy()
correction = hospital_correction.copy()
for i in tqdm(range(len(clean))):
# for i in tqdm(tax_error):
    for j in range(clean.shape[1]):
        dirty_cell = dirty.iloc[i,j]
        clean_cell = clean.iloc[i,j]
        correct_cell = correction.iloc[i,j]
        if(correct_cell!=dirty_cell):
            All_Fixed_Error += 1
        if(clean_cell!=dirty_cell):
            All_Data_Error += 1
            if(correct_cell==clean_cell or correct_cell in clean_cell):
                Correct_Fixed_Error += 1
Precision_hospital = Correct_Fixed_Error / All_Fixed_Error
Recall_hospital = Correct_Fixed_Error / All_Data_Error
F1_hospital = (2 * Precision_hospital * Recall_hospital) / (Precision_hospital + Recall_hospital)
Precision_hospital,Recall_hospital,F1_hospital

  0%|          | 0/1000 [00:00<?, ?it/s]

(0.9664694280078896, 0.962671905697446, 0.9645669291338582)

### Dataset Beers

In [3]:
test_file_path = 'inference/beer_test.csv' ## Replace the train file
beer_result = pd.read_csv(test_file_path,index_col=0)

count = 0
beer_clean = pd.read_csv('raw_dataset/DC/beers/original/clean.csv').fillna('')
beer_dirty = pd.read_csv('raw_dataset/DC/beers/original/dirty.csv').fillna('')
detector_beer = np.load('raw_dataset/DC/beers/detector/detector.npy')
beer_dirty.columns = beer_clean.columns
def try_convert_to_int(row):
    for x,y in row.items():
        if(x in ['ounces','ibu']):
            try:
                row[x] = int(y)
            except:
                row[x] = y
    return row
beer_clean = beer_clean.apply(try_convert_to_int,axis=1).astype(str)
beer_dirty = beer_dirty.apply(try_convert_to_int,axis=1).astype(str)
beer_correction = beer_dirty.copy()
for d in np.argwhere(detector_beer==1):
    i = d[0] 
    j = d[1] + 2
    
    try:
        predict = list(eval(beer_result.iloc[count,-1]).values())[0]
        beer_correction.iloc[i,j] = predict
        count += 1
    except:
        print(count)
        count += 1
# print(count,len(np.argwhere(detector_beer==1)),len(beer_result))
All_Data_Error = 0
All_Fixed_Error = 0
Correct_Fixed_Error = 0
for i in range(len(beer_clean)):
    for j in range(11):
        dirty_cell = beer_dirty.iloc[i,j]
        clean_cell = beer_clean.iloc[i,j]
        correct_cell = beer_correction.iloc[i,j]
        if(correct_cell!=dirty_cell):
            All_Fixed_Error += 1
        if(clean_cell!=dirty_cell):
            All_Data_Error += 1
            if(correct_cell==clean_cell):
                Correct_Fixed_Error += 1
Precision_hospital = Correct_Fixed_Error / All_Fixed_Error
Recall_hospital = Correct_Fixed_Error / All_Data_Error
F1_hospital = (2 * Precision_hospital * Recall_hospital) / (Precision_hospital + Recall_hospital)
Precision_hospital,Recall_hospital,F1_hospital

(0.9973190348525469, 0.9973190348525469, 0.9973190348525469)

### Dataset Rayyan

In [2]:
## Try to test recall on Rayyan Dataset
import json
test_file_path = 'inference_Transfer_ER/rayyan_train-MoE-Add--rayyan-test-20.csv'
result = pd.read_csv(test_file_path,index_col=0)

# result = pd.read_csv('inference_Transfer_ER/select--rayyan-test.csv',index_col=0)
rayyan_detector = np.load('raw_dataset/DC/rayyan/detector/detector.npy')
## If you run a different Error Detection result, change the 

rayyan_clean = pd.read_csv('raw_dataset/DC/rayyan/original/clean.csv').fillna('')
rayyan_dirty = pd.read_csv('raw_dataset/DC/rayyan/original/dirty.csv').fillna('')
def Str2Int(row):
    for index in range(11):
        temp = row[index]
        try:
            row[index] = str(int(temp))
        except:
            continue
    return row
rayyan_clean = rayyan_clean.apply(Str2Int,axis=1)
rayyan_dirty = rayyan_dirty.apply(Str2Int,axis=1)
count = 0
valid_count = 0
rayyan_correction = rayyan_dirty.copy()
import ast
for d in tqdm(np.argwhere(rayyan_detector==1)):
    i = d[0]
    j = d[1] + 1 ## Ignore Index
    try:
        predict = list(eval(result.iloc[count,-1]).values())[0]
        rayyan_correction.iloc[i,j] = predict
        valid_count += 1
    except:
        # print(result.iloc[count,-1])
        predict = result.iloc[count,-1]
        rayyan_correction.iloc[i,j] = predict
    count += 1
All_Data_Error = 0
All_Fixed_Error = 0
Correct_Fixed_Error = 0
clean = rayyan_clean.copy()
dirty = rayyan_dirty.copy()
correction = rayyan_correction.copy()
for i in tqdm(range(len(clean))):
# for i in tqdm(tax_error):
    for j in range(clean.shape[1]):
        dirty_cell = dirty.iloc[i,j]
        clean_cell = clean.iloc[i,j]
        correct_cell = correction.iloc[i,j]
        if(correct_cell!=dirty_cell):
            All_Fixed_Error += 1
        if(clean_cell!=dirty_cell):
            All_Data_Error += 1
            if(correct_cell==clean_cell or correct_cell in clean_cell):
                Correct_Fixed_Error += 1
Precision_hospital = Correct_Fixed_Error / All_Fixed_Error
Recall_hospital = Correct_Fixed_Error / All_Data_Error
F1_hospital = (2 * Precision_hospital * Recall_hospital) / (Precision_hospital + Recall_hospital)
Precision_hospital,Recall_hospital,F1_hospital

/tmp/ipykernel_27622/2627275201.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp = row[index]
/tmp/ipykernel_27622/2627275201.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  row[index] = str(int(temp))
/tmp/ipykernel_27622/2627275201.py:11: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  temp = row[index]
/tmp/ipykernel_27622/2627275201.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecate

  0%|          | 0/1117 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

(0.783001808318264, 0.9135021097046413, 0.8432327166504382)

### Column Type Annotation

### SimTab

In [ ]:
CUDA_VISIBLE_DEVICES=4,5,6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/CTA/CTA_SimTab_train_init --gpu_num 4 --file dataset/CTA/SimTab_test_few.json --max_token 128 --json

In [ ]:
CUDA_VISIBLE_DEVICES=4,5,6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/CTA/CTA_WebTable_train_init --gpu_num 4 --file dataset/CTA/WebTable_Test_few.json --max_token 128 --json

In [7]:
test_file_path = 'inference/SimTab_test_few.csv'
all_relation = np.load('raw_dataset/CTA/SimTab/sim_all_relation.npy',allow_pickle=True)
relation_dict = {}
for i in range(len(all_relation)):
    relation_dict[all_relation[i]] = i
SimTab_test = pd.read_csv(test_file_path,index_col=0)
def Ast(row):
    truth = list(eval(row['output']).values())[0]
    try:
        pred = list(eval(row['predict']).values())[0]
    except:
        pred = ''
    index = row['index']
    return truth, pred, index
SimTab_test_Transform = SimTab_test.apply(Ast,axis=1,result_type='expand')
SimTab_test_Transform.columns = ['truth','pred','index']
count = 0
truth_list = []
pred_list = []
for i in range(len(SimTab_test_Transform['index'].unique())):
    select_df = SimTab_test_Transform[SimTab_test_Transform['index'] == i]
    truth = select_df.iloc[0,0]
    select_df_filter = select_df[select_df['pred'].isin(all_relation)]
    try:
        pred = select_df_filter['pred'].value_counts().idxmax()
    except:
        pred = select_df['pred'].value_counts().idxmax()
    truth_list.append(truth)
    pred_list.append(pred)
    if truth==pred:
        count += 1
    # else:
    #     print(truth,select_df['pred'].value_counts().idxmax())
# count / len(SimTab_test_Transform['index'].unique()) ## 
SimTab_F1 = pd.DataFrame()
SimTab_F1['pred_output'] = pred_list
SimTab_F1['truth_output'] = truth_list
from sklearn.metrics import f1_score
pred = SimTab_F1['pred_output'].map(relation_dict).fillna(0).to_list()
truth = SimTab_F1['truth_output'].map(relation_dict).to_list()
f1_score(y_pred=pred,y_true=truth,average='micro'),f1_score(y_pred=pred,y_true=truth,average='macro')

(0.8935611038107752, 0.761725380779237)

### WebTable

In [22]:
test_file_path = 'inference/WebTable_Test_few.csv'
all_relation = np.load('raw_dataset/CTA/WebTable/webtable_all_relation.npy',allow_pickle=True)
relation_dict = {}
for i in range(len(all_relation)):
    relation_dict[all_relation[i]] = i
SimTab_test = pd.read_csv(test_file_path,index_col=0)
def Ast(row):
    truth = list(eval(row['output']).values())[0]
    try:
        pred = list(eval(row['predict']).values())[0]
    except:
        pred = ''
    index = row['index']
    return truth, pred, index
SimTab_test_Transform = SimTab_test.apply(Ast,axis=1,result_type='expand')
SimTab_test_Transform.columns = ['truth','pred','index']
count = 0
truth_list = []
pred_list = []
for i in range(len(SimTab_test_Transform['index'].unique())):
    select_df = SimTab_test_Transform[SimTab_test_Transform['index'] == i]
    truth = select_df.iloc[0,0]
    select_df_filter = select_df[select_df['pred'].isin(all_relation)]
    try:
        pred = select_df_filter['pred'].value_counts().idxmax()
    except:
        pred = select_df['pred'].value_counts().idxmax()
    truth_list.append(truth)
    pred_list.append(pred)
    if truth==pred:
        count += 1
    # else:
    #     print(truth,select_df['pred'].value_counts().idxmax())
# count / len(SimTab_test_Transform['index'].unique()) ## 
SimTab_F1 = pd.DataFrame()
SimTab_F1['pred_output'] = pred_list
SimTab_F1['truth_output'] = truth_list
from sklearn.metrics import f1_score
pred = SimTab_F1['pred_output'].map(relation_dict).fillna(0).to_list()
truth = SimTab_F1['truth_output'].map(relation_dict).to_list()
f1_score(y_pred=pred,y_true=truth,average='micro'),f1_score(y_pred=pred,y_true=truth,average='macro')

(0.9643463497453311, 0.7877228958461471)

## Evaluation on Relation Extraction

- RE have multiple correct choices, so we only evalutate precision.

In [ ]:
CUDA_VISIBLE_DEVICES=2,3 python vllm_inference_mistral_api.py --directory lora_weight/Expert/RE/RE-MoE-Add --gpu_num 2 --file dataset/RE/RE-test.json --max_token 128 --json

In [19]:
test_file_path = 'inference/RE-test.csv'
# all_relation = np.load('raw_dataset/RE/',allow_pickle=True)
RE_test = pd.read_csv('raw_dataset/RE/RE_test.csv',index_col=0) ## Multiple-Choice Ground Truth
SimTab_test = pd.read_csv(test_file_path,index_col=0)
SimTab_test['index'] = SimTab_test.index
def Ast(row):
    truth = list(eval(row['output']).values())[0]
    try:
        pred = list(eval(row['predict']).values())[0]
    except:
        pred = ''
    index = row['index']
    return truth, pred, index
SimTab_test_Transform = SimTab_test.apply(Ast,axis=1,result_type='expand')
SimTab_test_Transform.columns = ['truth','pred','index']
count = 0
truth_list = []
pred_list = []
for i in range(len(SimTab_test_Transform['index'].unique())):
    select_df = SimTab_test_Transform[SimTab_test_Transform['index'] == i]
    # truth = select_df.iloc[0,0]
    truth = eval(RE_test.iloc[i,1])
    select_df_filter = select_df[select_df['pred'].isin(all_relation)]
    try:
        pred = select_df_filter['pred'].value_counts().idxmax()
    except:
        pred = select_df['pred'].value_counts().idxmax()
    truth_list.append(truth)
    pred_list.append(pred)
    if truth.__contains__(pred):
        count += 1
    # else:
    #     print(truth,select_df['pred'].value_counts().idxmax())
count / len(SimTab_test_Transform['index'].unique())


0.900096525096525

## Evaluation for Schema Matching

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/SM/CMS_train-MoE-Add --gpu_num 2 --file dataset/SM/CMS-test.json --max_token 32 --json --guided_choices

In [ ]:
CUDA_VISIBLE_DEVICES=4,5 python vllm_inference_mistral_api.py --directory lora_weight/Expert/SM/synthea_train-MoE-Add --gpu_num 2 --file dataset/SM/synthea-test.json --max_token 32 --json 

In [26]:
# test_file_path = 'inference/CMS-test.csv' ## replace with the inference file
test_file_path = 'inference/synthea-test.csv' ## replace with the inference file

result_merge = pd.read_csv(test_file_path,index_col=0).fillna('')
def Transfer(row):
    if(row['output'].__contains__('mismatch')) or (row['output'].__contains__('dismatch')):
        label = 0
    else:
        label = 1
    if(row['predict'].__contains__('mismatch')) or (row['predict'].__contains__('dismatch')):
        predict = 0
    else:
        predict = 1
    return label,predict
result_output = result_merge.apply(Transfer,axis=1,result_type='expand')
from sklearn.metrics import f1_score, precision_score,recall_score
print(precision_score(y_true=result_output[0],y_pred=result_output[1]),recall_score(y_true=result_output[0],y_pred=result_output[1]),f1_score(y_true=result_output[0],y_pred=result_output[1]))

0.4838709677419355 0.7142857142857143 0.5769230769230769


## Evaluation for Data Imputation

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/DI/restaurant_train-MoE-Add --gpu_num 2 --file dataset/DI/restaurant_test_output_wide.json --max_token 128 --json

In [ ]:
CUDA_VISIBLE_DEVICES=6,7 python vllm_inference_mistral_api.py --directory lora_weight/Expert/DI/walmart_train-MoE-Add --gpu_num 2 --file dataset/DI/walmart_test_output_wide.json --max_token 128 --json

### Restaurant
- we strip the string and blank at the beginning/end, and replace certain synonym, e.g. `new york city` and `new york`

In [32]:
import re
def Restaurant_string_clean(text):
    text =  text.replace('new york city','new york')
    pattern = r"^[^a-zA-Z]*(.*?)\s*'$"
    result = re.match(pattern, text)
    if result:
        cleaned_text = result.group(1)
        return cleaned_text  
    else:
        return text

In [37]:
# train_file_path = 'inference/restaurant_test_output_wide.csv'
train_file_path = 'inference/walmart_test_output_wide.csv'
result = pd.read_csv(train_file_path,index_col=0)
# walmart_DI_result[walmart_DI_result['output']!=walmart_DI_result['predict']]
import ast
def AST(row):
    output = list(ast.literal_eval(row['output']).values())[0]
    output = Restaurant_string_clean(output)
    try:
        predict = list(ast.literal_eval(row['predict'].strip()).values())[0]
        predict = Restaurant_string_clean(predict)
    except:
        print(row['predict'])
        predict  = ''
    return output,predict
walmart_DI_ast = result.apply(AST,axis=1,result_type='expand')
len(walmart_DI_ast[walmart_DI_ast[0]==walmart_DI_ast[1]]) / len(walmart_DI_ast)

0.875

## Evaluation for Atttibute Value Extraction

In [38]:
ave_train = pd.read_csv('inference/oa_mine-test.csv',index_col=0)
import ast
def AST(row):
    output = row['output'].strip()
    predict = row['predict'].strip()
    output_item = list(ast.literal_eval(output).values())[0]
    predict_item = list(ast.literal_eval(predict).values())[0]
    row['output_item'] = output_item.lower()
    row['predict_item'] = predict_item.lower()
    return row
ave_train = ave_train.apply(AST,axis=1)
len(ave_train[ave_train['output_item']==ave_train['predict_item']]) / len(ave_train)

0.8049775601795186